<a href="https://colab.research.google.com/github/savpatpanda/protein/blob/master/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/savpatpanda/protein.git
%cd protein
!ls 
!pip3 install -r requirements.txt
# to push to GitHub, File > Save a copy in GitHub

Cloning into 'protein'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 48 (delta 21), reused 27 (delta 8), pack-reused 0
Unpacking objects: 100% (48/48), done.
/content/protein
data  notebook.ipynb  plotting.py  proteinModel.py  requirements.txt
     |████████████████████████████████| 1.8MB 6.2MB/s 
     |████████████████████████████████| 11.6MB 3.2MB/s 
     |████████████████████████████████| 14.5MB 301kB/s 
     |████████████████████████████████| 2.2MB 43.8MB/s 
     |████████████████████████████████| 296kB 50.0MB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation

In [26]:
from gemmi import cif 

try:
  doc = cif.read_file("./data/185472.cif")
  block = doc.sole_block()
  for item in block:
    if item.pair: pass#print(item.pair)
    if item.loop: 
      loop = item.loop
      for i in loop.tags:
        print(i)
        print(list(block.find_loop(i)))
      print()
except Exception as e:
  print(e)

_citation_id
['1']
_citation_doi
['10.1039/b203379h']
_citation_year
['2003']

_atom_type_symbol
['C', 'H', 'O']
_atom_type_description
['C', 'H', 'O']
_atom_type_scat_length_neutron
['6.646', '-3.739', '5.803']
_atom_type_scat_source
["'International Tables Vol C Table 4.4.4.1'", "'International Tables Vol C Table 4.4.4.1'", "'International Tables Vol C Table 4.4.4.1'"]

_symmetry_equiv_pos_as_xyz
["'x, y, z'", "'-x, y+1/2, -z+1/2'", "'-x, -y, -z'", "'x, -y-1/2, z-1/2'"]

_atom_site_label
['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'O1', 'O2', 'O3', 'O4', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8']
_atom_site_type_symbol
['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'O', 'O', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
_atom_site_fract_x
['0.1537(4)', '0.2473(4)', '0.2998(4)', '0.2609(4)', '0.1686(4)', '0.1158(4)', '0.0903(4)', '0.3674(4)', '0.3993(5)', '0.1235(5)', '0.0098(6)', '0.2886(5)', '0.4029(6)', '0.3698(11)', '0.3016(12)', '0.1406(11)', '0.0433(10)', '0.4